<a href="https://colab.research.google.com/github/Kunal-1669/deep_learning_Final_Project/blob/main/deep_learning_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset
data=load_dataset('lavita/ChatDoctor-HealthCareMagic-100k')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/112165 [00:00<?, ? examples/s]

In [5]:
import pandas as pd
df=pd.DataFrame(data['train'])

In [6]:
df

,instruction,input,output
0,"If you are a doctor, please answer the medical...",I woke up this morning feeling the whole room ...,"Hi, Thank you for posting your query. The most..."
1,"If you are a doctor, please answer the medical...",My baby has been pooing 5-6 times a day for a ...,Hi... Thank you for consulting in Chat Doctor....
2,"If you are a doctor, please answer the medical...","Hello, My husband is taking Oxycodone due to a...","Hello, and I hope I can help you today.First, ..."
3,"If you are a doctor, please answer the medical...",lump under left nipple and stomach pain (male)...,HI. You have two different problems. The lump ...
4,"If you are a doctor, please answer the medical...",I have a 5 month old baby who is very congeste...,Thank you for using Chat Doctor. I would sugge...
...,...,...,...
112160,"If you are a doctor, please answer the medical...",im 25 years old ..i started using mtp kit on 5...,"Hello, Thanks for letting us know your health ..."
112161,"If you are a doctor, please answer the medical...",My 5 year old son has been coughing for a mont...,As you have mentioned in your history that you...
112162,"If you are a doctor, please answer the medical...",My toes on right foot more than left are numb ...,Hi. The numbness and blue discoloration could ...
112163,"If you are a doctor, please answer the medical...","I was diagnosis with pleurisy last Tuesday, an...",Thanks for your question on Chat Doctor. Treat...


In [7]:
df.shape


(112165, 3)

In [8]:
df = df.drop('instruction', axis=1)

In [9]:
df

,input,output
0,I woke up this morning feeling the whole room ...,"Hi, Thank you for posting your query. The most..."
1,My baby has been pooing 5-6 times a day for a ...,Hi... Thank you for consulting in Chat Doctor....
2,"Hello, My husband is taking Oxycodone due to a...","Hello, and I hope I can help you today.First, ..."
3,lump under left nipple and stomach pain (male)...,HI. You have two different problems. The lump ...
4,I have a 5 month old baby who is very congeste...,Thank you for using Chat Doctor. I would sugge...
...,...,...
112160,im 25 years old ..i started using mtp kit on 5...,"Hello, Thanks for letting us know your health ..."
112161,My 5 year old son has been coughing for a mont...,As you have mentioned in your history that you...
112162,My toes on right foot more than left are numb ...,Hi. The numbness and blue discoloration could ...
112163,"I was diagnosis with pleurisy last Tuesday, an...",Thanks for your question on Chat Doctor. Treat...


In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df['input'], df['output'], test_size=0.2, random_state=42)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt

In [11]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
df[['input', 'output']] = df[['input', 'output']].apply(lambda x: x.apply(lambda y: word_tokenize(y.lower())))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
df

,input,output
0,"[i, woke, up, this, morning, feeling, the, who...","[hi, ,, thank, you, for, posting, your, query,..."
1,"[my, baby, has, been, pooing, 5-6, times, a, d...","[hi, ..., thank, you, for, consulting, in, cha..."
2,"[hello, ,, my, husband, is, taking, oxycodone,...","[hello, ,, and, i, hope, i, can, help, you, to..."
3,"[lump, under, left, nipple, and, stomach, pain...","[hi, ., you, have, two, different, problems, ...."
4,"[i, have, a, 5, month, old, baby, who, is, ver...","[thank, you, for, using, chat, doctor, ., i, w..."
...,...,...
112160,"[im, 25, years, old, .., i, started, using, mt...","[hello, ,, thanks, for, letting, us, know, you..."
112161,"[my, 5, year, old, son, has, been, coughing, f...","[as, you, have, mentioned, in, your, history, ..."
112162,"[my, toes, on, right, foot, more, than, left, ...","[hi, ., the, numbness, and, blue, discoloratio..."
112163,"[i, was, diagnosis, with, pleurisy, last, tues...","[thanks, for, your, question, on, chat, doctor..."


In [14]:
def remove_stop_words(tokens):
    stop_words_set = set(stopwords.words('english'))
    return [word for word in tokens if word.lower() not in stop_words_set]


In [15]:
from nltk.corpus import stopwords
df['input'] = df['input'].apply(remove_stop_words)
df['output'] = df['output'].apply(remove_stop_words)

# Display the DataFrame
print(df)

                                                    input  \
0       [woke, morning, feeling, whole, room, spinning...   
1       [baby, pooing, 5-6, times, day, week, ., last,...   
2       [hello, ,, husband, taking, oxycodone, due, br...   
3       [lump, left, nipple, stomach, pain, (, male, )...   
4       [5, month, old, baby, congested, terrible, cou...   
...                                                   ...   
112160  [im, 25, years, old, .., started, using, mtp, ...   
112161  [5, year, old, son, coughing, month, ., taken,...   
112162  [toes, right, foot, left, numb, turning, blue,...   
112163  [diagnosis, pleurisy, last, tuesday, ,, still,...   
112164  [within, past, hours, ,, husband, developed, r...   

                                                   output  
0       [hi, ,, thank, posting, query, ., likely, caus...  
1       [hi, ..., thank, consulting, chat, doctor, ., ...  
2       [hello, ,, hope, help, today.first, ,, medicat...  
3       [hi, ., two, differ

In [16]:
from nltk.stem import PorterStemmer

def stem_tokenized_text(tokenized_text):
    # Initialize the PorterStemmer
    stemmer = PorterStemmer()

    # Perform stemming on each token
    stemmed_tokens = [stemmer.stem(token) for token in tokenized_text]

    return stemmed_tokens

In [17]:
df['input'] = df['input'].apply(stem_tokenized_text)
df['output'] = df['output'].apply(stem_tokenized_text)

In [18]:
import spacy

# Load spaCy model
nlp = spacy.load('en_core_web_sm')



In [19]:
# Function to apply word embeddings using spaCy
def get_word_embeddings(word_list):
    text = ' '.join(word_list)
    doc = nlp(text)
    return doc.vector

In [ ]:
df['WordEmbeddings_input'] = df['input'].apply(get_word_embeddings)

In [ ]:
df['WordEmbeddings_output'] = df['output'].apply(get_word_embeddings)

In [ ]:
csv_path = '/content/sample_data/my_dataframe.csv'  # Specify your desired path
df.to_csv(csv_path, index=False)

# Download the CSV file
from google.colab import files
files.download(csv_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>